<div style="
    background: linear-gradient(to right, #001f3f, #0074D9, #7FDBFF);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #00ccff, 0px 0px 20px #00ccff;
    box-shadow: 0px 0px 15px rgba(0, 204, 255, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    🚀 Welcome to AI Recognition & Gesture Control! 🎥
</div>

<div style="
    background: linear-gradient(to right, #2c3e50, #4ca1af);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #00ff99, 0px 0px 20px #00ff99;
    box-shadow: 0px 0px 15px rgba(0, 255, 153, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    🎭 Real-Time AI Filters & Object Detection! 🤖
</div>

<div style="
    background: linear-gradient(to right, #8e44ad, #9b59b6);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #ff66ff, 0px 0px 20px #ff66ff;
    box-shadow: 0px 0px 15px rgba(255, 102, 255, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    🕵️‍♂️ AI That Sees & Reacts Instantly! 🔥
</div>

In [1]:
import cv2
import time
import numpy as np
import torch
import mediapipe as mp
import threading

# 🚀 --- Load YOLOv5 Model for Object Detection ---
# We are using the pre-trained 'yolov5s' model from Ultralytics
# YOLOv5 (You Only Look Once) is a popular real-time object detection model.
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# 🎯 --- Initialize Mediapipe for Hand & Pose Detection ---
# Mediapipe is used to perform hand and pose detection.
# Hands: Detects and tracks hand landmarks.
# Pose: Detects full-body pose landmarks.
mp_hands = mp.solutions.hands  
mp_pose = mp.solutions.pose  
mp_draw = mp.solutions.drawing_utils  # Drawing utility to visualize landmarks

# Configure the hand detection model
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
# Configure the pose detection model
pose = mp_pose.Pose()

# 📷 --- Initialize Webcam Capture ---
cap = cv2.VideoCapture(0)  # Open the default webcam (index 0)

# ⏱️ --- FPS Control to Avoid Overload ---
FPS_LIMIT = 12  # Set the maximum FPS to avoid overloading the CPU
last_time = time.time()  # Track the last processed frame time for FPS regulation

# 🔥 --- Feature Toggles ---
ai_glow = False  # Toggle AI glow effect (press 'b' to toggle)
particles_enabled = False  # Toggle particle effect (press 'v' to toggle)
split_screen = False  # Toggle mirror effect (press 'c' to toggle)

# 🎨 --- Image Filters Dictionary ---
# A dictionary of different filters you can apply to the webcam feed
def apply_filter(frame, mode):
    """Applies different visual effects based on selected mode."""
    filters = {
        1: lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),  # Grayscale 🌑
        2: lambda img: cv2.Canny(img, 50, 150),  # Edge Detection ✏️
        3: lambda img: cv2.GaussianBlur(img, (15, 15), 0),  # Blur 🌫️
        4: lambda img: cv2.bitwise_not(img),  # Inverted Colors 🔄
        5: lambda img: cv2.applyColorMap(img, cv2.COLORMAP_JET),  # Heatmap 🔥
        6: lambda img: cv2.applyColorMap(img, cv2.COLORMAP_HOT),  # Thermal Effect ♨️
        7: lambda img: cv2.applyColorMap(img, cv2.COLORMAP_OCEAN),  # Ocean Effect 🌊
        8: lambda img: cv2.stylization(img, sigma_s=60, sigma_r=0.2),  # Cartoon Effect 🎭
        9: lambda img: cv2.detailEnhance(img, sigma_s=5, sigma_r=0.1),  # Detail Enhancement 🔍
    }
    return filters.get(mode, lambda x: x)(frame)  # Default: No filter

# 💡 --- AI Glow Effect ---
def apply_ai_glow(frame):
    """Creates a futuristic AI glow effect using edge detection & color mapping."""
    small = cv2.resize(frame, (320, 240))  # Reduce resolution for faster processing
    blurred = cv2.GaussianBlur(small, (15, 15), 0)  # Smooth effect
    edges = cv2.Canny(small, 20, 80)  # Extract edges from the image
    edges_colored = cv2.applyColorMap(edges, cv2.COLORMAP_WINTER)  # Colorize edges for visual effect
    return cv2.addWeighted(cv2.resize(blurred, (frame.shape[1], frame.shape[0])), 0.7,
                           cv2.resize(edges_colored, (frame.shape[1], frame.shape[0])), 0.3, 0)

# 🌌 --- Particle System (Dynamic) ---
class Particle:
    """Represents a particle that moves dynamically and reacts to hand gestures."""
    def __init__(self, width, height):
        self.x = np.random.randint(0, width)  # Random initial X position
        self.y = np.random.randint(0, height)  # Random initial Y position
        self.vx = np.random.uniform(-1.5, 1.5)  # Random X velocity
        self.vy = np.random.uniform(-1.5, 1.5)  # Random Y velocity
        self.size = np.random.randint(3, 6)  # Particle size
        self.color = (0, 255, 0)  # Green particles

    def update(self, target_x=None, target_y=None):
        """Moves towards a target position (index finger tip) with a smooth effect."""
        if target_x and target_y:
            dx, dy = target_x - self.x, target_y - self.y
            dist = max(np.sqrt(dx**2 + dy**2), 1)  # Avoid division by zero
            self.vx += (dx / dist) * 0.6  # Attraction force
            self.vy += (dy / dist) * 0.6

        # Apply movement with limits
        self.x = np.clip(self.x + self.vx, 0, 640)
        self.y = np.clip(self.y + self.vy, 0, 480)

    def draw(self, frame):
        """Draws the particle on the frame."""
        cv2.circle(frame, (int(self.x), int(self.y)), self.size, self.color, -1)

particles = [Particle(640, 480) for _ in range(100)]  # Create 100 particles

# 🚀 --- YOLO Object Detection (Runs in Parallel) ---
yolo_results = None
def yolo_thread(frame):
    """Runs YOLO object detection in a separate thread for performance optimization."""
    global yolo_results
    yolo_results = model(frame)

# 🔄 --- Main Loop ---
filter_mode = 0  # Default filter mode

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 🕒 --- FPS Regulation (Prevents Overload) ---
    if (time.time() - last_time) < 1 / FPS_LIMIT:
        continue
    last_time = time.time()

    # 🎭 --- Convert Frame for MediaPipe Processing ---
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 📦 --- Start YOLO in a Separate Thread ---
    yolo_task = threading.Thread(target=yolo_thread, args=(frame,))
    yolo_task.start()

    # 🖐 --- Hand Tracking ---
    hand_x, hand_y = None, None
    hand_results = hands.process(rgb_frame)
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            hand_x = int(hand_landmarks.landmark[8].x * frame.shape[1])  # Index finger tip X
            hand_y = int(hand_landmarks.landmark[8].y * frame.shape[0])  # Index finger tip Y

    # 🏃 --- Pose Detection (Full-Body Tracking) ---
    pose_results = pose.process(rgb_frame)
    if pose_results.pose_landmarks:
        mp_draw.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                               mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3))

    # 📌 --- YOLO Object Detection Results ---
    if yolo_results:
        for det in yolo_results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = det.tolist()
            if conf > 0.5:  # Only detect objects with confidence > 50%
                label = f"{model.names[int(cls)]} {conf:.2f}"
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # --- Apply AI Glow Effect ---
    if ai_glow:
        frame = apply_ai_glow(frame)

    # --- Apply Selected Filter ---
    if not ai_glow:
        frame = apply_filter(frame, filter_mode)

    # --- Apply Particles ---
    if particles_enabled:
        for particle in particles:
            particle.update(hand_x, hand_y)
            particle.draw(frame)

    # 🪞 --- Apply Mirror Reflection Effect (Split Screen) ---
    if split_screen:
        flipped = cv2.flip(frame, 1)  # Mirror Effect
        frame[:, frame.shape[1] // 2:] = flipped[:, frame.shape[1] // 2:]

    # 🖥 --- Display Frame ---
    cv2.imshow("AI Gesture Filters & Object Detection & Pose Detection", frame)

    # --- Debugging Output ---
    print(f"Modes -> Glow: {ai_glow}, Particles: {particles_enabled}, Mirror: {split_screen}, Filter: {filter_mode}")

    # 🎮 --- Keyboard Controls ---
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC to exit
        break
    elif key == ord("b"):  # Toggle AI Glow
        ai_glow = not ai_glow
    elif key == ord("v"):  # Toggle Particles
        particles_enabled = not particles_enabled
        particles = [Particle(640, 480) for _ in range(100)]
    elif key == ord("c"):  # Toggle Mirror Effect
        split_screen = not split_screen
    elif key in range(48, 58):  # Keys 0-9 for Filters
        filter_mode = key - 48

# 🧹 --- Cleanup ---
cap.release()  # Release the webcam capture
cv2.destroyAllWindows()  # Close all OpenCV windows

Using cache found in C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-13 Python-3.12.3 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0


C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: False, Mirror: False, Filter: 0


C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0


C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0


C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\GAMING/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, Mirror: False, Filter: 0
Modes -> Glow: False, Particles: True, M

<div style="
    background: linear-gradient(to right, #ff5733, #c70039);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #ff9966, 0px 0px 20px #ff9966;
    box-shadow: 0px 0px 15px rgba(255, 153, 102, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    ✅ AI Recognition Completed! What's Next? 🚀
</div>


<div style="
    background: linear-gradient(to right, #f39c12, #e67e22);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #ffcc00, 0px 0px 20px #ffcc00;
    box-shadow: 0px 0px 15px rgba(255, 204, 0, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    📸 Save Your AI-Enhanced Screenshots Now! 📂
</div>

<div style="
    background: linear-gradient(to right, #16a085, #1abc9c);
    padding: 15px;
    border-radius: 20px;
    text-align: center;
    font-size: 36px;
    font-family: 'Orbitron', sans-serif;
    color: white;
    text-shadow: 0px 0px 10px #00ffcc, 0px 0px 20px #00ffcc;
    box-shadow: 0px 0px 15px rgba(0, 255, 204, 0.6);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;">
    🔥 Stay Tuned! More AI Features Coming Soon! ⚡
</div>